# Mystery box - 2 - Create dataset for distance detector
### Dennis Bakhuis - 10th November 2022
### https://linkedin.com/in/dennisbakhuis/



In [ ]:
from random import sample
from pathlib import Path
import json
import shutil

import cv2
import matplotlib.pyplot as plt
from PIL import Image

from tqdm.auto import tqdm

import pandas as pd
import pigeonXT as pixt

from IPython.display import display
from IPython.display import Image as dimage

from datasets import load_dataset

import numpy as np

from joblib import Parallel, delayed

## Open video file and get fps

In [ ]:
image_file = '../data/raw/Session22/VID_20221019_171145.mp4'
cap = cv2.VideoCapture(image_file)

In [ ]:
fps_precise = cap.get(cv2.CAP_PROP_FPS)
fps = round(fps_precise)

print(f"FPS of video: {fps} ({fps_precise})")

## Extract frames at frame rate of 1 FPS

This takes a while (about 3min17 on my computer):

In [ ]:
%%time

count, current_time = 0, 0
frames, times = [], []

# cap is a generator and can only be iterated once
with tqdm() as pbar:
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
            times.append(current_time)
            count += fps
            current_time += fps_precise
            cap.set(cv2.CAP_PROP_POS_FRAMES, count)
            pbar.update(1)

        else:
            cap.release()
            break

In [ ]:
print(f"Frames selected: {len(frames)}")

In [ ]:
frame = frames[1000]

fig, ax = plt.subplots(figsize=(8, 4))
_ = ax.imshow(frame)

## Create function to crop frames to LCD screen

In [ ]:
crop_x = slice(100,990)
crop_y = slice(720,1020)

def crop_flip(image):
    return image[crop_y, crop_x]


In [ ]:
im = crop_flip(frames[0])

print(im.shape)
fig, ax = plt.subplots(figsize=(3, 12))
_ = ax.imshow(im)

## Convert images to model size / extract distance
The cropping part is slightly larger as there is some 
shaking. This makes some artifacts enter the frame from
the equal sign or the symbols above.

In [ ]:
def prep_image(image):
    # crop image and to grayscale
    image = crop_flip(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # center selection in new zeros image
    new_image = np.zeros([480, 360])
    new_image[175:300, 30:330] = image[155:280, 487:787]

    # threshold image
    _, new_image = cv2.threshold(
        new_image, 200, 255, cv2.THRESH_BINARY,
    )
    
    # invert, i.e. black on white
    new_image = 255 - new_image
    
    return new_image

image = frames[200].copy()

print(image.shape)
fig, ax = plt.subplots(figsize=(7, 6))
_ = ax.imshow(prep_image(image), cmap='gray')

In [ ]:
processed_frames = [
    prep_image(image)
    for image in tqdm(frames)
]

In [ ]:
plt.imshow(processed_frames[2000])

## Save images images

In [ ]:
# Create folder if not exist
dataset_path = Path('../data/mystery_box/images_unlabeled')
if not dataset_path.exists():
    dataset_path.mkdir(parents=True)

In [ ]:
def store_frame(ix, frame):
    image = Image.fromarray(frame)    
    with open(dataset_path / f"{str(ix).zfill(4)}.png", 'wb') as f:
        image.convert("L").save(f, format='png')


_ = Parallel(n_jobs=6)(
    delayed(store_frame)(ix, frame) 
    for ix, frame in tqdm(enumerate(processed_frames), total=len(processed_frames))
)

## Label 250 random frames

In [ ]:
files = list(Path('../data/mystery_box/images_unlabeled').glob('*.png'))

In [ ]:
N = 250
subset = sample(files, N)

In [ ]:
annotations = pixt.annotate(
  subset,
  task_type="captioning",
  display_fn=lambda filename: display(dimage(filename))
)

In [ ]:
metadata = [
    {
        "file_name": row.example.name,
        "text": json.dumps({"distance": row.label}),
    } for row in annotations.itertuples(index=False) 
]

In [ ]:
metadata = sorted(metadata, key=lambda x: int(x['file_name'].split('.')[0]))

In [ ]:
with open('../data/mystery_box/metadata.jsonl', 'w') as f:
    for row in metadata:
        f.write(f"{json.dumps(row)}\n")

## Create Huggingface style dataset from labeled images

In [ ]:
with open('../data/mystery_box/metadata.jsonl', 'r') as f:
    metadata = [
        json.loads(row)
        for row in f.readlines()
    ]

In [ ]:
source_path = Path('../data/mystery_box/images_unlabeled')
sink_path = Path('../data/mystery_box/images')

if not sink_path.exists():
    sink_path.mkdir()

In [ ]:
# Copy files to be labeled
for row in metadata:
    source_file = source_path / row['file_name']
    if source_file.exists():
        source_file.rename(sink_path / row['file_name'])

In [ ]:
# copy labeled data into images folder
shutil.copy('../data/mystery_box/metadata.jsonl', sink_path)

In [ ]:
dataset = load_dataset(
    "imagefolder",
    data_dir = "../data/mystery_box/images",
    split="train",
)
    

In [ ]:
len(dataset)

In [ ]:
dataset[0]

In [ ]:
dataset[0]['image'].resize([60, 100])

## Store dataset

In [ ]:
# dataset.save_to_disk("../data/mystery_box/distance_detector")
dataset.push_to_hub("bakhuisdennis/mystery_box")

## Load the dataset directly from the hub

In [ ]:
from datasets import load_dataset
ds = load_dataset("bakhuisdennis/mystery_box")